In [1]:
import os
import dotenv

dotenv.load_dotenv()
os.chdir(os.environ["PYTHONPATH"])

# load the embedding model

In [2]:
import torch
from transformers import (
    T5EncoderModel,
    T5Tokenizer,
    pipeline
)

In [3]:
device = torch.device("cuda:0")

print(device, "is used")

model_name = "Rostlab/prot_t5_xl_uniref50"

tokenizer = T5Tokenizer.from_pretrained(model_name, do_lower_case=False)
model = T5EncoderModel.from_pretrained(model_name).to(device)

# if device == torch.device("cpu"):
    # model.to(torch.float32)

embeddings_pipeline = pipeline("feature-extraction", model=model, tokenizer=tokenizer)

cuda:0 is used


You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Device set to use cuda:0


In [13]:
def embed_sequences(row):
    row['sequence_embedding'] = embeddings_pipeline(row['sequence'])[0][0]
    return row

# calculate embeddings and perform cluster analysis

In [8]:
import pandas as pd
from datasets import Dataset

In [6]:
prot_df = pd.read_csv("data/similarity/proteins.csv")
prot_df

,hash,sequence,len,len_bin
0,f4700b879aef8c84b62418d2a47de698e9e331176c8579...,MSSSSPSSQSRLSVPGRTPHLPPLTIPHTVSAEGLATPNTPHALLP...,529,"(433.0, 689.0]"
1,14d37ac5624cf44e84d31c34be0c59931899daa3fdcd16...,PHRYRPGTVALREIRRYQKSTELLIRKLPFQRLVREIAQDFKTDLR...,756,"(689.0, 5890.0]"
2,39abc653d9fbeddc04309795feae21d6fb4b1c4d1c2d15...,MGKYTKTAILSVYDKTGLLDLAKGLVENNVRILASGGTANMVREAG...,591,"(433.0, 689.0]"
3,1e4cca78876ecb4b0683037381aa0b7c2ef252ab746871...,MLEICLKLVGCKSKKGLSSSSSCYLEEALQRPVASDFEPQGLSEAA...,1130,"(689.0, 5890.0]"
4,d09efb0ddd42cab0c93f2137c542a6da14eb19c5eec283...,MGETLGDSPVDPEHGAFADALPMSTSQEITMVDTEMPFWPTNFGIS...,505,"(433.0, 689.0]"
...,...,...,...,...
63189,9a122fc06407b99169e30f2e0f63dd0d076dd27a3f863b...,MTWRQAVLLSCFSAVVLLSMLREGTSVSVGTMQMAGEEASEDAKQK...,138,"(99.999, 278.0]"
63190,c422d561d3c69cf4fe601b522a664fe7934008e982df9e...,MAEPASVTVTFDDVALYFSEQEWEILEKWQKQMYKQEMKTNYETLD...,752,"(689.0, 5890.0]"
63191,231ee06744cadd3f373b33cdaf191a22f6e98e7fdee8ad...,MLVVQMPFSFPMAHFILFVFTVSTIFHVQQRLAKIQAMWELPVQIP...,343,"(278.0, 433.0]"
63192,74ab43e422350613e89be6f5fde367457228794e235520...,MANDTHTDDLDELLDSALDDFKDLNLTQRNGGVKKEEGDKKETESL...,245,"(99.999, 278.0]"


In [9]:
prot_dataset = Dataset.from_pandas(prot_df)

In [14]:
embed_prot_dataset = prot_dataset.map(embed_sequences)

Map:   0%|          | 0/63194 [00:00<?, ? examples/s]

In [17]:
embed_prot_dataset.to_parquet("data/similarity/protein_embeddings.parquet.gzip")

Creating parquet from Arrow format:   0%|          | 0/64 [00:00<?, ?ba/s]

559890659

In [ ]:
kmeans = KMeans()